# Result models

In [1]:
#import warnings
#warnings.filterwarnings('ignore')
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [2]:
import pandas as pd
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width', 10000)
import numpy as np
import yaml

from vessel_manoeuvring_models.visualization.plot import track_plot, plot
from phd.visualization.plot_ship import track_plots
from vessel_manoeuvring_models.prime_system import PrimeSystem
from phd.visualization.plot_prediction import plot_total_force, plot_force_components, predict, plot_compare_model_forces, plot_parameter_contributions, group_parameters, joined_parameter_contributions, same_ylims
from vessel_manoeuvring_models.symbols import *
import matplotlib.pyplot as plt
from vessel_manoeuvring_models.substitute_dynamic_symbols import run, lambdify, remove_functions, prime
from vessel_manoeuvring_models.parameters import df_parameters as df_parameters_all
p = df_parameters_all["symbol"]
import statsmodels.api as sm
import sympy as sp
from sympy import Eq, symbols, latex
import paper
from paper import save_fig, save_eq
import yaml
with open("styles.yml", mode='r') as file:
    styles = yaml.safe_load(file)
from phd.pipelines.predict.nodes import score
import vessel_manoeuvring_models.models.IMO_simulations
from phd.paper.equation import to_latex

In [3]:
#%reload_kedro
ship='wPCC'
ship_data = catalog.load(f"{ship}.ship_data")

loaders = {
        
    'Reference model' : catalog.load(f"{ship}.models_VCT_MDL")['semiempirical_covered'],
    'Physics informed ID' : catalog.load(f"{ship}.models_ID_hull2")['semiempirical_covered'],
    
    #'Abkowitz  VCT': catalog.load(f"{ship}.models_rudder_VCT_MDL")['Abkowitz'],
    'Abkowitz ID' : catalog.load(f"{ship}.models_ID_hull_rudder2")['Abkowitz'],
}

[02/14/24 09:33:47] INFO     Loading data from 'wPCC.ship_data' (YAMLDataSet)...                ]8;id=737018;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=50673;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

                    WARNING  /home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/par ]8;id=135476;file:///home/maa/anaconda3/envs/phd/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=310401;file:///home/maa/anaconda3/envs/phd/lib/python3.10/warnings.py#109\109]8;;\
                             titioned_dataset.py:200: KedroDeprecationWarning: 'PartitionedDataset'                
                             has been moved to `kedro-datasets` and will be removed in Kedro                       
                             0.19.0.                                                                               
                               warnings.warn(                                                                      
                                                                                                                   

                    WARNING  /home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/__i ]8;id=207406;file:///home/maa/anaconda3/envs/phd/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=120383;file:///home/maa/anaconda3/envs/phd/lib/python3.10/warnings.py#109\109]8;;\
                             nit__.py:44: KedroDeprecationWarning: 'AbstractDataSet' has been                      
                             renamed to 'AbstractDataset', and the alias will be removed in Kedro                  
                             0.19.0                                                                                
                               return getattr(kedro.io.core, name)                                                 
                                                                                                                   

                    INFO     Loading data from 'wPCC.models_VCT_MDL' (PartitionedDataset)...    ]8;id=743688;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=298956;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

                    INFO     Loading data from 'wPCC.models_ID_hull2' (PartitionedDataset)...   ]8;id=188052;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=631132;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

                    INFO     Loading data from 'wPCC.models_ID_hull_rudder2'                    ]8;id=839934;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=625163;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\
                             (PartitionedDataset)...                                                               

In [4]:
models = {name:loader() for name,loader in loaders.items()}

In [5]:
dofs=['X','Y','N']
_=[]
for model_name,model in models.items():
    pars_all = {}
    for dof in dofs:
        pars = {key:value for key,value in model.parameters.items() if ((value!=0) and (key[0]==dof) and (not 'dot' in key) and (not 'delta' in key) and (not 'thrust' in key))}
        pars_all.update(pars)
    
    pars_all['model'] = model_name
    _.append(pars_all)

df_parameters = pd.DataFrame(_)
df_parameters = df_parameters.set_index('model').transpose()
df_parameters.index = pd.Series(df_parameters.index).apply(lambda x:fr"${{{x[0]}'}}_{{{x[1:]}}}$")
df_parameters.index.name='Coefficient'

In [6]:
df_parameters

model,Reference model,Physics informed ID,Abkowitz ID
Coefficient,,,
${X'}_{0}$,-0.001025,-0.001025,-0.001025
${X'}_{u}$,0.000336,0.000336,0.000336
${X'}_{vv}$,-0.000483,-0.003830,0.004788
${X'}_{rr}$,-0.000303,0.000994,0.012427
${X'}_{vr}$,0.002451,0.003034,0.019839
${Y'}_{v}$,-0.007978,-0.008948,-0.020783
${N'}_{v}$,-0.003209,-0.002781,-0.000983
${N'}_{r}$,-0.001552,-0.001534,-0.003246


In [7]:
df_parameters.to_csv(paper.file_path_with_nb_ref("parameters.csv", directory='tables'), index=True)

## Other parameters

In [8]:
keys = list(pars_all.keys())
keys.remove('model')
keys

['Xrr', 'Xvr', 'Xvv', 'X0', 'Xu', 'Yv', 'Nr', 'Nv']

In [9]:
model = models['Reference model']
other_parameters = pd.Series(model.parameters)
other_parameters = other_parameters.drop(keys);

## Added masses

In [10]:
mask = other_parameters.index.str.contains('dot')
added_masses = other_parameters[mask]
other_parameters = other_parameters.drop(added_masses.index);
added_masses

Xudot    0.000179
Yvdot   -0.006109
Yrdot   -0.000303
Nvdot   -0.000128
Nrdot   -0.000157
dtype: float64

In [11]:
pd.DataFrame(added_masses).transpose().to_csv(paper.file_path_with_nb_ref("added_masses.csv", directory='tables'), index=False)

In [12]:
df_added_masses = pd.DataFrame(index=added_masses.index)
df_added_masses['Parameter'] = df_parameters_all.loc[added_masses.index]['symbol']
df_added_masses['Parameter'] = df_added_masses['Parameter'].apply(lambda x: f"${x}$")
df_added_masses['Value'] = added_masses
df_added_masses = df_added_masses.set_index('Parameter').transpose()

In [13]:
df_added_masses

Parameter,$X_{\dot{u}}$,$Y_{\dot{v}}$,$Y_{\dot{r}}$,$N_{\dot{v}}$,$N_{\dot{r}}$
Value,0.000179,-0.006109,-0.000303,-0.000128,-0.000157


In [14]:
df_added_masses.to_csv(paper.file_path_with_nb_ref("added_masses.csv", directory='tables'), index=False)

In [15]:
symbols = []
for key, eq in model.equations.items():
    symbols+=list(eq.rhs.free_symbols)
symbols = list(set(symbols))

In [16]:
parameters = {}
for symbol in symbols:
    key = str(symbol)
    if key in model.parameters:
        parameters[symbol] = model.parameters[key]  

    if key in model.ship_parameters:
        parameters[symbol] = model.ship_parameters[key]    

In [17]:
parameters

{C_L_tune: 1.0,
 delta_alpha_s: 0,
 A_R_U: 0.01647301230087661,
 x: 0.027800910486710073,
 s: -10,
 gamma_0_port: 0.044,
 r_0: 0.06067961165048543,
 e_0: 0.9,
 A_R: 0.030095732397021395,
 y_p_port: -0.2176,
 y_p_stbd: 0.2176,
 b_R: 0.17427184466019416,
 delta_lim: 0.2617993877991494,
 kappa_v_gamma_g: 0.12614619635893423,
 kappa_v: 0.8875332235434824,
 kappa_r_gamma_g: 0.8265178168548643,
 nu: 1.18849e-06,
 c_t: 0.19878640776699028,
 w_f: 0.297,
 c_r: 0.14660194174757282,
 C_D_tune: 1.0685595467405165,
 a_H: 0.12824552804749745,
 C_D0_tune: 7.740218543115913,
 rho: 1000,
 Omega: 0,
 y_R: 0,
 A_R_C: 0.013622720096144783,
 x_R: -2.45,
 kappa_r: 1.194039652718919,
 x_H: 0.12495489987343933,
 gamma_0_stbd: -0.044}

In [18]:
df_parameters = pd.DataFrame(index=parameters.keys())
df_parameters['Parameter'] = pd.Series(df_parameters.index, index=df_parameters.index).apply(lambda x : f"${to_latex(x)}$")
df_parameters.sort_values(by='Parameter', inplace=True)
df_parameters['Value'] = parameters
df_parameters.index = pd.Series(df_parameters.index).astype(str)

df_parameters.drop(index=['delta_alpha_s'], inplace=True)  # remove stalling
df_parameters.drop(index=['y_R'], inplace=True)  # remove stalling
#df_parameters.reset_index(inplace=True, drop=True)

In [19]:
df_parameters

,Parameter,Value
A_R_C,$A_{R C}$,0.013623
A_R_U,$A_{R U}$,0.016473
A_R,$A_{R}$,0.030096
C_D_tune,$C_{D tune}$,1.068560
C_D0_tune,$C_{D0 tune}$,7.740219
C_L_tune,$C_{L tune}$,1.000000
Omega,$\Omega$,0.000000
delta_lim,$\delta_{lim}$,0.261799
gamma_0_port,$\gamma_{0 port}$,0.044000
gamma_0_stbd,$\gamma_{0 stbd}$,-0.044000


In [20]:
df_parameters.loc['delta_lim','Parameter'] = r'$\delta_{lim}$ $[rad]$'
df_parameters.loc['gamma_0_port','Parameter'] = r'$\gamma_{0port}$ $[rad]$'
df_parameters.loc['gamma_0_stbd','Parameter'] = r'$\gamma_{0stbd}$ $[rad]$'

In [21]:
table = paper.table_columns(df_parameters,columns=3)

In [22]:
table

,Parameter,Value,Parameter1,Value1,Parameter2,Value2
0,$A_{R C}$,0.013623,$\kappa_{r \gamma g}$,0.826518,$e_{0}$,0.900000
1,$A_{R U}$,0.016473,$\kappa_{r}$,1.194040,$r_{0}$,0.060680
2,$A_{R}$,0.030096,$\kappa_{v \gamma g}$,0.126146,$s$,-10.000000
3,$C_{D tune}$,1.068560,$\kappa_{v}$,0.887533,$w_{f}$,0.297000
4,$C_{D0 tune}$,7.740219,$\nu$,0.000001,$x$,0.027801
5,$C_{L tune}$,1.000000,$\rho$,1000.000000,$x_{H}$,0.124955
6,$\Omega$,0.000000,$a_{H}$,0.128246,$x_{R}$,-2.450000
7,$\delta_{lim}$ $[rad]$,0.261799,$b_{R}$,0.174272,$y_{p port}$,-0.217600
8,$\gamma_{0port}$ $[rad]$,0.044000,$c_{r}$,0.146602,$y_{p stbd}$,0.217600
9,$\gamma_{0stbd}$ $[rad]$,-0.044000,$c_{t}$,0.198786,NaN,NaN


In [23]:
table.to_csv(paper.file_path_with_nb_ref("other_parameters.csv", directory='tables'), index=False)

## Main particulars

In [32]:
ship_parameters = pd.Series(model.ship_parameters)
keys = list(set(ship_parameters.index) - set(df_parameters.index))
ship_parameters = ship_parameters[keys].copy()
ship_parameters.drop(index=[
    'TWIN',
    'y_fan_fore',
    'x_p',
    'y_R_stbd',
    'y_fan_aft',
    'x_r',
    'volume',
    'n_prop',
    'z_p',
    'x_fan_fore',
    'y_R_port',
    'A_YV',
    'A_XV',
    'rho_A',
    'x_fan_aft',
    'CB',
    'c',
    'w_p0',
    'z_R',
    'y_R',
    
], inplace=True)

In [33]:
ship_parameters

I_z             742.046279
scale_factor     41.200000
T                 0.206311
m               441.026784
B                 0.946602
D                 0.121359
tdf               0.117000
L                 5.014563
x_G              -0.239897
dtype: float64

In [34]:
df_ship_parameters = pd.DataFrame(index=ship_parameters.index)
df_ship_parameters['Parameter'] = pd.Series(df_ship_parameters.index, index=df_ship_parameters.index).apply(lambda x : f"${to_latex(x)}$")
df_ship_parameters['Value'] = ship_parameters
df_ship_parameters['Description'] = ''

In [35]:
df_ship_parameters.loc['I_z','Parameter'] = r'$I_{zz}$'
df_ship_parameters.loc['x_G','Parameter'] = r'$x_{G}$'
df_ship_parameters.loc['scale_factor','Parameter'] = r'Scale factor'
df_ship_parameters.loc['tdf','Parameter'] = r'$t_{df}$'
df_ship_parameters.loc['T','Parameter'] = r'$T_{m}$'
df_ship_parameters.sort_index(inplace=True)

In [36]:
df_ship_parameters.loc['B','Description'] = r'Breadth'
df_ship_parameters.loc['D','Description'] = r'Propeller diameter'
df_ship_parameters.loc['I_z','Description'] = r'Yaw moment of inertia around midship'
df_ship_parameters.loc['L','Description'] = r'Length between perpendiculars'
df_ship_parameters.loc['T','Description'] = r'Mean draught'
df_ship_parameters.loc['m','Description'] = r'Ship mass'
df_ship_parameters.loc['tdf','Description'] = r'Thrust deduction factor'
df_ship_parameters.loc['x_G','Description'] = r'Longitudinal c.o.g.'


In [37]:
df_ship_parameters

,Parameter,Value,Description
B,$\mathtt{\text{B}}$,0.946602,Breadth
D,$\mathtt{\text{D}}$,0.121359,Propeller diameter
I_z,$I_{zz}$,742.046279,Yaw moment of inertia around midship
L,$\mathtt{\text{L}}$,5.014563,Length between perpendiculars
T,$T_{m}$,0.206311,Mean draught
m,$\mathtt{\text{m}}$,441.026784,Ship mass
scale_factor,Scale factor,41.200000,
tdf,$t_{df}$,0.117000,Thrust deduction factor
x_G,$x_{G}$,-0.239897,Longitudinal c.o.g.


In [38]:
df_ship_parameters.to_csv(paper.file_path_with_nb_ref("main_particulars.csv", directory='tables'), index=False)

In [39]:
model.ship_parameters

{'T': 0.2063106796116504,
 'L': 5.014563106796117,
 'CB': 0.45034232324249973,
 'B': 0.9466019417475728,
 'rho': 1000,
 'x_G': -0.23989669902912625,
 'm': 441.0267843660858,
 'I_z': 742.0462794081202,
 'volume': 0.4410267843660858,
 'scale_factor': 41.2,
 'x_r': -2.45,
 'TWIN': 1,
 'x_p': -2.42219908951329,
 'y_p_port': -0.2176,
 'y_p_stbd': 0.2176,
 'z_p': 0.12730849514563097,
 'y_R_port': -0.2176,
 'y_R_stbd': 0.2176,
 'D': 0.12135922330097086,
 'w_p0': 0.151,
 'A_XV': 0.5938354227542653,
 'A_YV': 3.3958124999999995,
 'rho_A': 1.225,
 'n_prop': 2,
 'tdf': 0.117,
 'A_R': 0.030095732397021395,
 'b_R': 0.17427184466019416,
 'c_r': 0.14660194174757282,
 'c_t': 0.19878640776699028,
 'c': 0.17269417475728155,
 'x_fan_fore': 1.2877184466019416,
 'x_fan_aft': -1.5381815533980583,
 'y_fan_aft': 0,
 'y_fan_fore': 0,
 'r_0': 0.06067961165048543,
 'x': 0.027800910486710073,
 'x_R': -2.45,
 'y_R': 0,
 'z_R': 0,
 'w_f': 0.297,
 'A_R_C': 0.013622720096144783,
 'A_R_U': 0.01647301230087661}